Drag this into the _data folder or setd directories for the parquet files

In [51]:
import pandas as pd

bonds = pd.read_parquet("mergent_bond.parquet")
mapping = pd.read_parquet("RED_and_ISIN_mapping.parquet")

In [52]:
bonds

,offering_date,maturity,amount_outstanding,security_level,offering_yield,issuer_cusip,issue_cusip,treasury_spread,maturity_time_frame
12801,1996-11-18,2003-12-01,185000.0,SS,11.500,89558D,AA8,540.0,7
16755,1997-02-21,2004-03-01,172200.0,SEN,NaN,90467N,AE6,NaN,7
22121,1998-09-02,2005-04-15,99269.0,SENS,NaN,827499,AB0,NaN,6
32394,2000-12-19,2008-10-01,30296.0,SENS,NaN,421924,AP6,NaN,7
68631,2004-03-11,2014-04-08,19000.0,SEN,NaN,52517P,WB3,NaN,10
...,...,...,...,...,...,...,...,...,...
198180,2024-06-07,2026-06-12,250000.0,SEN,4.898,3133ER,HD6,NaN,2
198182,2024-06-18,2034-06-26,43000.0,SEN,4.539,3133ER,JC6,NaN,10
198183,2024-06-25,2028-03-28,50000.0,SEN,5.300,3135GA,TW1,NaN,3
198184,2024-06-05,2029-02-15,325000.0,SEN,NaN,054561,AN5,NaN,4


In [53]:
mapping.columns

Index(['redcode', 'ticker', 'obl_cusip', 'isin', 'tier'], dtype='object')

In [54]:
map_df = mapping[['redcode', 'obl_cusip']]
map_df.dropna(inplace=True)
map_df['issuer_cusip'] = map_df.apply(lambda row: row['obl_cusip'][:6], axis=1)

map_df

/var/folders/8f/m5g40vjs3yj876tchrr05l640000gn/T/ipykernel_12721/1192555507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_df.dropna(inplace=True)
/var/folders/8f/m5g40vjs3yj876tchrr05l640000gn/T/ipykernel_12721/1192555507.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_df['issuer_cusip'] = map_df.apply(lambda row: row['obl_cusip'][:6], axis=1)


,redcode,obl_cusip,issuer_cusip
0,001AEC,001957AV1,001957
1,001AEC,001957BC2,001957
2,001AEC,001957AP4,001957
3,001AEC,001957BD0,001957
4,002AA6,002824AK6,002824
...,...,...,...
10377,YZ79AI,Y1636YAG3,Y1636Y
10378,YZ79CL,G2154WAA0,G2154W
10384,YZ7C7O,G2233YAA1,G2233Y
10385,YZ9A6N,DUMYREFOB,DUMYRE


In [62]:
bonds_test = bonds.copy()

In [63]:
redcode_mapping = map_df.set_index("issuer_cusip")["redcode"].to_dict()

bonds_test["redcode"] = bonds["issuer_cusip"].map(redcode_mapping)
bonds_test = bonds_test.dropna(subset=['redcode', 'offering_yield', 'treasury_spread']).set_index('offering_date')
bonds_test.sort_index()

,maturity,amount_outstanding,security_level,offering_yield,issuer_cusip,issue_cusip,treasury_spread,maturity_time_frame,redcode
offering_date,,,,,,,,,
2005-01-04,2010-01-27,1100000.0,SEN,4.34400,52517P,YN5,64.0,5,576D98
2010-08-03,2020-08-15,122900.0,SEN,6.87500,74153Q,AH5,396.8,10,7B5AC5
2014-07-28,2024-08-01,350000.0,SEN,3.49607,902133,AR8,58.0,10,LUCAAI
2014-07-29,2024-08-01,300000.0,SEN,4.07094,26885B,AA8,160.0,10,UW8EGE
2014-08-05,2025-02-15,418917.0,SEN,3.38505,20030N,BL4,90.0,10,2C033N
...,...,...,...,...,...,...,...,...,...
2023-09-06,2034-03-15,500000.0,SEN,6.09254,361448,BM4,180.0,10,3J758C
2023-10-05,2033-11-15,725000.0,SEN,5.91861,049560,AZ8,120.0,10,04DEB6
2024-01-02,2029-01-05,650000.0,SEN,4.87711,26441C,CC7,95.0,5,28A85D


count       0
unique      0
top       NaN
freq      NaN
Name: isin, dtype: object